# Future roll scenario : Market Activity Approach - C#

### Overview
Future contracts are liquid and often used to gain exposure to a variety of asset classes. However they must be rolled on a periodic basis to avoid expiry or triggering delivery.

The period surrounding contract rollover can be challenging time. Future roll strategies help to better prepare moves in the market and optimize trading positions.

In order to identify future roll optimal dates, the following methods can be used:
1. Maturity date
2. Trading volume
3. Market activity

In this example, the strategy chosen to determine the future roll date  is based on the **market activity**. 
Indeed, as number of ticks within roll period are typically split between the expiring contract and the new contracts, it can used as an indicator to track market activity and large price swings and high spreads.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** bars for a <i>`given future, all maturities`</i>.

### Inputs/outputs
Future activity-based roll sample requires future contact's identifier and date time intervals as per inputs. It returns future's continuous price over the look back period.<br>

This sample shows also how to plot the returned bars using an open source library.
### Services used
This sample uses gRPC requests in order to retrieve information from hosted services. The queried endpoint in this script are:
* *StaticDataService*: to get the intrument identifier from the input
* Topologies: in order to retrieve the market activity (daily tick count) for selected maturities
* *TickBarsService*: to get tick bars data for each instrument

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Topology.V1*
    * *Systemathics.Apis.Services.StaticData.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run future roll sample - market activity approach

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Topology.V1;
using Systemathics.Apis.Services.TickAnalytics.V1;
using Systemathics.Apis.Services.StaticData.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Protobuf;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using System.Collections.Generic;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Select futures scope

#### 3.1 Define inputs
In a first time the user has to decide the lookup time span and the selected future contract.

In [ ]:
// Define start, end and future contract
var start = new DateTime(2020,02, 01);
var end = new DateTime(2020, 07, 31);
var contract = "WBS"; // WTI Crude future

#### 3.2 Select future contracts

In [ ]:
// instantiate the static data service
var service = new StaticDataService.StaticDataServiceClient(channel);

In [ ]:
// Get futures from static data request
// We chose to retrieve the selected instrument with its contract code.
var request = new StaticDataRequest
{
    AssetType = AssetType.Future,
    FutureContract = contract,
    Count = 1000
};

// store futures and display results
var reply = service.StaticData(request, headers);
var futures = reply.Futures.OrderBy(future => future.Maturity.Year).ThenBy(future => future.Maturity.Month); 

In [ ]:
// Display results
//display(futures)

#### 3.3 Filter futures matching input period

In [ ]:
// Filter futures : select only futures with a maturity matching the selected period
var tmpList = new List<FutureEntry>();
var lastFuture = true; // we need this boolean to select one more future
foreach (var future in futures) {
    var maturity = new DateTime(future.Maturity.Year,future.Maturity.Month, future.Maturity.Day);
    if (maturity > start) 
    {
        if (maturity < end) {
            tmpList.Add(future);
        } 
        else 
        {
            if (lastFuture == true) 
            {
                tmpList.Add(future); // add the last contract
                lastFuture = false;
            } 
            else {
                break;
            }
        }
    }
}
var count = tmpList.Count;
var selectedFutures = tmpList.ToArray();

In [ ]:
// Display previously selected futures
// display(selectedFutures)

### Step 4: Determine roll date based on market activity


#### 4.1 Get market activity for each selected maturity
Market activity is computed by calling the *tick topologies service* which returns the tick count for a chosen *time granularity*. The following code snippet is a method to handle *tick toplogies requests* creation to retrieve ticks count on daily basis.

In [ ]:
// Define a method that creates a topologies request for a given instrument to batch request later in the sample
public static TopologiesRequest GetTopologiesRequest(Identifier identifier)
{
    return new TopologiesRequest 
    { 
        Identifier = identifier, 
        Granularity = TopologyGranularity.Daily,
        Level = Systemathics.Apis.Type.Shared.V1.Level.TradesAndBook
    };    
}

In [ ]:
// Define a method to handle tick toplogies reply and create a dictionary of tick count per date
// Ex: 2016-06-18 | 55
//     2016-06-19 | 35 
public static Dictionary<string, double> GetTicks(IEnumerable<TopologyEntry> entries) 
{
    var dict = new Dictionary<string, double>();
    foreach (var entry in entries) {
        var count = entry.TicksCount;
        var time = new DateTime(entry.Begin.Year, entry.Begin.Month, entry.Begin.Day);
        var key = time.ToString("yyyy-MM-dd");
        dict[key] = count;
    }
    return dict;
}

In [ ]:
// Intiate dictionary to handle ticks count for each future contract
// Keys = tickers
// Values = dictionaries of tick count per date
var ticktionary = new Dictionary<string, Dictionary<string, double>>();

// Instantiate the tick topoliges service
var topologiesService = new TopologiesService.TopologiesServiceClient(channel);

// Process the tick toplogies reply: store ticks count for each future contract
foreach (var elt in selectedFutures)
{
    // Generate the tick topoliges request
    var topologiesRequest = GetTopologiesRequest(elt.Identifier);
    
    var topologiesReply = topologiesService.Topologies(topologiesRequest, headers);
    var key = elt.Identifier.Ticker;
    ticktionary[key] = GetTicks(topologiesReply.Entries);
}

In [ ]:
// Display ticktionary keys: selected future contacts' tickers
var keys = ticktionary.Keys;
//display(ticktionary.Keys)

In [ ]:
// Display ticktionary 1st value: daily ticks count for the 1st future contract
var sample = ticktionary[keys.First()];
//display(sample)

#### 4.2 Determine roll dates
The following code snippets creates methods to determine *front and back* future contracts for a given date.

In [ ]:
// Define a method that gets front ticker according to the given list of futures
// Input: 
//   -Ticker/Maturity dictionary
//   -current date
// Output: 
//   -front ticker (can be null)
public static string GetFrontTicker(Dictionary<string, DateTime> tickerMaturityPair, DateTime currentDate) 
{
    foreach (var kvp in tickerMaturityPair) 
    {
        // get maturity and compare to currentTime
        var maturity = kvp.Value; 
        if (maturity> currentDate) {
            return kvp.Key;
        }
    }
    return null;
}

In [ ]:
// Define a method that gets back ticker according to the given list of futures
// Input: 
//   -Ticker/Maturity dictionary
//   -current date
// Output: 
//   -back ticker (can be null)
public static string GetBackTicker(Dictionary<string, DateTime> tickerMaturityPair, DateTime currentDate) 
{
    var front = false; // check if we 'passed' front
    foreach (var kvp in tickerMaturityPair) 
    {
        // get maturity and compare to currentTime
        var maturity = kvp.Value; 
        if (maturity> currentDate) {
            if (front) {
                return kvp.Key;
            }
            front = true;
        }
    }
    return null;
}

The following code snippet creates a dictionary of *ticker* and *future maturity* pairs. This represents the input of the previous methods.

In [ ]:
// Store maturities in a ticker indexed dictionary
var tickerMaturities = new Dictionary<string, DateTime>();
foreach (var future in selectedFutures) {
    var key = future.Identifier.Ticker;
    var date = new DateTime(future.Maturity.Year,future.Maturity.Month,future.Maturity.Day,0,0,0 );
    tickerMaturities[key]= date;
}
display(tickerMaturities)

The following code snippet gives the *front and back future contracts* on daily basis over the chosen look back period.

In [ ]:
var test = start;
while (test < end) {
    var d = test.ToString("yyyy-MM-dd");
    Console.WriteLine($"{d} F: {GetFrontTicker(tickerMaturities,test)}, B {GetBackTicker(tickerMaturities,test)}");
    test = test.AddDays(1);
}

#### 4.3 Prepare bars request parameters
Select start date, end date and instrument identifier for the previsouly selected futures.

In [ ]:
// Prepare start and end dates for upcoming tick bars requests
var barsStarts =  new Dictionary<string, DateTime>();
var barsEnds = new Dictionary<string, DateTime>();

The code snippet below generates, for each instrument included in the given period, the **start date** and **end date** necessary for upcoming bars request. Those are computed by comparing ticks count of *front* and *back* contract.

Day after day, the algorithm retrieves, if any, the **number of ticks** for both back and front. 

Roll date is determined when **#ticks_front $\le$ #ticks_back**. Start and end dates for bars request are then generated.


In [ ]:
// Initializing
var currentDate = start;
var previousFrontTicker = GetFrontTicker(tickerMaturities, currentDate);
barsStarts[previousFrontTicker] = currentDate;

// Increment for each date within the chosen period
while(currentDate < end) {
    var frontTicker = GetFrontTicker(tickerMaturities, currentDate);
    if (frontTicker != previousFrontTicker) {
        barsStarts[frontTicker] = currentDate;
    }
    previousFrontTicker = frontTicker;
    var backTicker = GetBackTicker(tickerMaturities, currentDate);
    if (frontTicker == null) {
        Console.WriteLine($"End: should not occur, handled in while loop");
        break;
    }
    if (backTicker == null) // Case where we reached end date
    {
        barsEnds[frontTicker] = end;
        break;
    }
    
    // Get ticks count dictionary for each ticker 
    var frontTopologies = ticktionary[frontTicker];
    var backTopologies = ticktionary[backTicker];

    // -->  Check if we have tick size for front and back
    // Front    Back     Behavior
    //   V       V    => compare sizes
    //   V       X    => skip to next current date
    //   X       V    => ROLL
    //   X       X    => skip to next current date

    var currentDateString = currentDate.ToString("yyyy-MM-dd");
    
    // Both ticks => compare
    if (frontTopologies.ContainsKey(currentDateString) && backTopologies.ContainsKey(currentDateString)) 
    {
        var backSize = backTopologies[currentDateString];
        var frontSize = frontTopologies[currentDateString];
        if (frontSize < backSize) {
            // ROLL
            Console.WriteLine($"Rolling from {frontTicker} to {backTicker}: {currentDate} - {frontSize} vs {backSize}");
            barsEnds[frontTicker] = tickerMaturities[frontTicker]; // Add end for bars request
            currentDate = tickerMaturities[frontTicker].AddDays(1); // Skipping to another day
            continue;
        } else {
            // SKIP
            currentDate = currentDate.AddDays(1);// Skipping day to day
        continue;
        }
    }
    
    // Only back ticks => roll
    if (!frontTopologies.ContainsKey(currentDateString) && backTopologies.ContainsKey(currentDateString)) 
    {
        // ROLL
        Console.WriteLine($"Rolling from {frontTicker} to {backTicker}: {currentDate} -  no more front ticks");
        barsEnds[frontTicker] = tickerMaturities[frontTicker]; // Add end for bars request
        currentDate = tickerMaturities[frontTicker].AddDays(1); // Skipping to another day
        continue;
    }
    
    // Only front ticks => skip
    if (frontTopologies.ContainsKey(currentDateString) && !backTopologies.ContainsKey(currentDateString)) 
    {
        currentDate = currentDate.AddDays(1);// Skipping day to day
        continue;
    }
    
    // No front or back ticks => skip
    if (!frontTopologies.ContainsKey(currentDateString) && !backTopologies.ContainsKey(currentDateString)) 
    {
        currentDate = currentDate.AddDays(1); // Skipping day to day
        continue;
    }
}

In [ ]:
foreach (var f in selectedFutures) {
    display(f.Identifier.Ticker);
}

The following code snippet generates *tick bars* requests' parameters:

In [ ]:
// Create a list of tuples containing the instrument identifier and date intervals for bars request
// The days delta chosen here defines the delay until expiration before rolling
var barsRequestData = new List<Tuple<Identifier, DateTime,DateTime>>();

if (barsEnds.Keys.Count != barsStarts.Keys.Count) 
{
    Console.WriteLine($"Different start/end size");
}
var count = barsEnds.Keys.Count;
var keys = barsEnds.Keys.ToArray();
for (var i =0; i< count; i++) {
    var identifier = selectedFutures.Select(f => f.Identifier).Where(f=>f.Ticker == keys[i]).Single();
    var start = barsStarts[keys[i]];
    var end = barsEnds[keys[i]];
    var data = new Tuple<Identifier, DateTime,DateTime>(identifier, start, end);
    barsRequestData.Add(data);
}

In [ ]:
// Display bars request parameters
//display(barsRequestData)

### Step 5: Get bars to get a continuous price

#### 5.1 Define bar request creation method
The following code snippt defines a method that creates and handles *tick bars* requests: 

In [ ]:
public static TickBarsRequest GetBarsRequest(Identifier identifier, DateTime start, DateTime end)
{
    // Set the bar duration (1 hour bars in seconds)
    var sampling = 60 * 60;

    // Set the bar calculation field (Trade price)
    var field = BarPrice.Trade; 

    // Begin
    var beginTime = new TimeOfDay { Hours = 00, Minutes = 00, Seconds = 00 };

    // Create bars request data intervals
    var dateIntervals = new DateInterval()
      {
        StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
        EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }

      };
    
    // Generate constraints based on the previous date selection
    var constraints = new Constraints(); 
    constraints.DateIntervals.Add(dateIntervals);
    
    // Generate tick bars request
    var request = new TickBarsRequest 
    {
        Identifier = identifier, 
        Constraints = constraints,  
        Sampling = new Duration {Seconds = sampling},
        Field = field
    };
    return request;
}

#### 5.2 Process bars request for each future
We have to use the previously defined  `GetBarsRequest` method to generate the request and process it. Results will be stored in a dictionary before being displayed.

In [ ]:
// Intiate dictionary to handle bars data for each future contract
var barsResponses = new Dictionary<string, List<TickBarsResponse>>();

// Instantiate the tick bars service
var barsService = new TickBarsService.TickBarsServiceClient(channel);

// Process the tick bars reply: store tick bars for each future contract
foreach (var elt in barsRequestData)
{
    // Generate tick bars request
    var barsRequest = GetBarsRequest(elt.Item1, elt.Item2, elt.Item3);
    
    var key = elt.Item1.Ticker;
    barsResponses[key] = new List<TickBarsResponse>();
    
    var call = barsService.TickBars(barsRequest, headers);
    await foreach (var bar in call.ResponseStream.ReadAllAsync())
    {
        barsResponses[key].Add(bar);
    }
}

In [ ]:
// Verification: check first future bars
//display(barsResponses[barsResponses.Keys.First()])

### Step 6: Visualize data

The following code snippets retrieve for each maturity, the *Close prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.

In [ ]:
// Define a method transforming a dictionary key/value pair to a graph content
public static Scattergl ToGraph(KeyValuePair<string, List<TickBarsResponse>> kvp)
{
    return new Scattergl
    {
        name = kvp.Key,
        x = kvp.Value.Select(b => b.TimeStamp.ToDateTime()).ToArray(),
        y = kvp.Value.Select(b => b.Close).ToArray(),
    };
}

In [ ]:
var chartContent = barsResponses.Select(ToGraph).ToArray();
var chart = Chart.Plot(chartContent);
var layout = new Layout.Layout(){barmode = "group" };
chart.WithLayout(layout);
chart.WithTitle($"{contract} roll using market activity indicator | 1-hour bars [{start:yyyy-MM-dd} - {end:yyyy-MM-dd}] ");
chart.WithXTitle("Dates");
chart.WithYTitle("Close price");
chart.WithLegend(true);
chart.Width = 1500;
chart.Height = 800;
display(chart);